In [10]:
from sklearn.svm import SVC
from gensim.models.doc2vec import Doc2Vec
import re
import jieba
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib 
import random

In [11]:
divorce_model=Doc2Vec.load("divorce/divorce.model")
loan_model=Doc2Vec.load("loan/loan.model")
labor_model=Doc2Vec.load("labor/labor.model")

In [12]:
data_input = {1: "divorce/data.txt", 2: "labor/data.txt", 3: "loan/data.txt"}
models={1:divorce_model,2:loan_model,3:labor_model}
label_size = 20
test_ratio = 0.3
punction = "！？。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."

In [13]:
def line_processing(line):  # 提取每行数据的文本内容
    line = line.strip().split('\t')
    sentence = line[1]
    sentence = re.sub(r'[{}]'.format(punction),' ',sentence).split(' ')
    sent=[]
    for sub_sentence in sentence:
        sent.extend(list(jieba.cut(sub_sentence)))
    return line[0], sent, line[2]

In [14]:
def constructDataSet(data_path,model_tag): #构建X，Y的数据集
    data_file = open(data_path,'r',encoding='utf-8')
    lines = data_file.read().splitlines()
    X=[]
    Y=[]
    d2v = models[model_tag]
    for line in lines:
        _,x,y = line_processing(line)
        x=d2v.infer_vector(x)
        X.append(x)
        y = list(map(int,y.split()))
        Y.append(y)
    Y = np.array(Y).transpose()
    return X,Y

In [15]:
def splitDataSet(X,Y): #构建训练集和测试集
    X_train,X_test,Y_train,Y_test =  train_test_split(X,Y,test_size=test_ratio,shuffle=True)
    return X_train,X_test,Y_train,Y_test

In [16]:
def trainSVM(X,Y,model_path): #训练单个分类器
    X_train,X_test,Y_train,Y_test = splitDataSet(X,Y)
    classifier = SVC(gamma='auto')
    classifier.fit(X=X_train,y=Y_train)
    accuracy = classifier.score(X_test,Y_test)
    joblib.dump(classifier,model_path)
    print(model_path,accuracy)

In [17]:
def beginTrain():
    for i in range(1,4):
        X,Y = constructDataSet(data_input[i],i)
         tag = list(range(len(Y)))
        random.shuffle(tag)
        for j in tag:
            model_path = data_input[i].split("/")[0]+'/'+str(j+1)+".model"
            trainSVM(X,Y[j],model_path)

In [18]:
beginTrain()

divorce/7.model 0.9691956645750143
divorce/17.model 0.995436394751854
divorce/4.model 0.9458071876782659
divorce/8.model 0.9697661152310325
divorce/2.model 0.8727895037079293
divorce/3.model 0.8921848260125499
divorce/19.model 0.995436394751854
divorce/10.model 0.9734740444951512
divorce/16.model 0.9942954934398175
divorce/12.model 0.9854535082715346
divorce/1.model 0.7957786651454649
divorce/20.model 0.9937250427837993
divorce/14.model 0.9960068454078722
divorce/13.model 0.9883057615516257
divorce/6.model 0.9666286366229321
divorce/11.model 0.9928693667997718
divorce/9.model 0.9671990872789503
divorce/5.model 0.9569309754706218
divorce/18.model 0.9925841414717627
divorce/15.model 0.9945807187678266
labor/16.model 1.0
labor/2.model 0.9495601173020528
labor/3.model 0.9536656891495601
labor/8.model 0.9794721407624634
labor/6.model 0.9648093841642229
labor/12.model 0.9982404692082112
labor/5.model 0.9665689149560117
labor/11.model 0.9982404692082112
labor/7.model 0.9747800586510263
labor/

ValueError: The number of classes has to be greater than one; got 1 class